# DL-модель для классификации состояния дерева (Good/Fair/Poor) по данным из NY 2015 Street Tree Census.

## Импорт датасета 2015 Street Tree Census - Tree Data

Описание датасета https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh/about_data

In [6]:
import os
import zipfile
import requests

# URL для загрузки датасета
url = "https://www.kaggle.com/api/v1/datasets/download/new-york-city/ny-2015-street-tree-census-tree-data"
output_path = os.path.expanduser("./ny-2015-street-tree-census-tree-data.zip")

# Загрузка файла
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=128):
            f.write(chunk)
    print(f"Файл загружен: {output_path}")
else:
    print(f"Ошибка при загрузке файла: {response.status_code}")

# Распаковка архива
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.expanduser("./ny-2015-street-tree-census-tree-data"))
    print("Файлы распакованы")

# Указание пути к конкретному файлу
file_path = os.path.expanduser("./ny-2015-street-tree-census-tree-data/2015_Street_Tree_Census_-_Tree_Data.csv")

# Проверка, существует ли файл по указанному пути
if os.path.exists(file_path):
    # Чтение данных в DataFrame
    df = pd.read_csv(file_path)
    # Просмотр первых строк датасета
    print(df.head())
else:
    print(f"Файл не найден по пути: {file_path}")

Файл загружен: ./ny-2015-street-tree-census-tree-data.zip
Файлы распакованы
Файл не найден по пути: ./ny-2015-street-tree-census-tree-data/2015_Street_Tree_Census_-_Tree_Data.csv
